In [1]:
import json
import copy
import re
from watson_developer_cloud import AssistantV1

In [2]:
# load workspace from json
with open("data/workspace.json", mode='r') as file:
        workspace = json.load(file)

In [ ]:
# load workspace form credentials
creds = {'username':'{username}',
         'password': '{password}',
         'version':'2019-02-28',
         'url': '{url}'}

service = AssistantV1(username=creads['username'],
                      password=creds['password'],
                      version=creds['version'],
                      url=creds['url'])
workspace_id = ''
workspace = service.get_workspace(workspace_id = workspace_id, export=True).get_result()

In [3]:
condition_list = []
analysis_results_list = []
#workspace = None
intents = []
entities = []
dialog_nodes = []
if 'intents' in workspace:
    intents_list = copy.deepcopy(workspace['intents'])
    for intent in intents_list:
        intents.append('#' + intent.get('intent', None))
    intents_set = set(intents)
        
if 'entities' in workspace:
    entities_list = copy.deepcopy(workspace['entities'])
    for entity in entities_list:
        entities.append('@' + entity.get('entity', None))
    entities_set = set(entities)
    
if 'dialog_nodes' in workspace:
    dialog_nodes = copy.deepcopy(workspace['dialog_nodes'])
    for dialog_node in dialog_nodes:
        if 'conditions' in dialog_node:
            condition_list.append((dialog_node['dialog_node'], dialog_node['conditions']))

In [4]:
# BROKEN INTENTS and NOT USED INTENTS
dialog_node_intents = []
for node, condition in condition_list:
    intent_list = re.findall(r'#[a-zA-z0-9_]*',condition)
    dialog_node_intents.append((node, intent_list))

# check if all intents in dialog_node_intents are exists in intents_set
problematic_dialog_node_intents = []
not_used_intents = set()
for dialog_node, dialog_node_intents_list in dialog_node_intents:
    for node_intent in dialog_node_intents_list:
        if node_intent not in intents_set:
            problematic_dialog_node_intents.append((dialog_node, node_intent))
        else:
            not_used_intents.add(node_intent)
            
not_used_intents = intents_set - not_used_intents
not_used_intents = list(sorted(not_used_intents))
# analysis_results_list = [{'analysis_name': "Broken intents in nodes conditions analysis", 'description': "intent from a condition not found in workspace's intents", 'result_data': problematic_dialog_node_intents}]
analysis_results_list.append({'analysis_name': "Broken intents in nodes conditions analysis", 'description': "intent from a condition not found in workspace's intents", 'result_data': problematic_dialog_node_intents})
analysis_results_list.append({'analysis_name': "Not used intents in nodes conditions analysis", 'description': "intent from a condition not found in workspace's intents", 'result_data': not_used_intents})

In [5]:
# BROKEN ENTITIES
dialog_node_entities = []
for node, condition in condition_list:
    entity_list = re.findall(r'@[a-zA-z0-9_]*', condition)
    dialog_node_entities.append((node, entity_list))

# chech if all entities in dialog_node_entities are exists in entities_set
problematic_dialog_node_entities = []
for dialog_node, dialog_node_entities_list in dialog_node_entities:
    for node_entity in dialog_node_entities_list:
        if node_entity not in entities_set:
            problematic_dialog_node_entities.append((dialog_node, node_entity))

analysis_results_list.append({'analysis_name': "Broken entities in nodes conditions analysis", 'description': "entity from a condition not found in workspace's entities", 'result_data': problematic_dialog_node_entities})

In [6]:
print("-------------------RESULTS-------------------")
for count, analysis_errors in enumerate(analysis_results_list):
    
    print("analysis number ", count+1)
    print("analysis name: ", analysis_errors['analysis_name'])
    print("description: ", analysis_errors['description'])
    print("n. errors: ", len(analysis_errors['result_data']))
    for error in analysis_errors['result_data']:
        print("ERROR: ", error)
    print('-'*40)

-------------------RESULTS-------------------
analysis number  1
analysis name:  Broken intents in nodes conditions analysis
description:  intent from a condition not found in workspace's intents
n. errors:  0
----------------------------------------
analysis number  2
analysis name:  Not used intents in nodes conditions analysis
description:  intent from a condition not found in workspace's intents
n. errors:  7
ERROR:  #decision_replies
ERROR:  #interface_interactions
ERROR:  #interface_issues
ERROR:  #navigation
ERROR:  #positive_reaction
ERROR:  #selections
ERROR:  #system_reliance
----------------------------------------
analysis number  3
analysis name:  Broken entities in nodes conditions analysis
description:  entity from a condition not found in workspace's entities
n. errors:  21
ERROR:  ('node_6_1488752733255', '@sys')
ERROR:  ('node_4_1489442065546', '@sys')
ERROR:  ('node_4_1489442065546', '@sys')
ERROR:  ('node_3_1488775609591', '@sys')
ERROR:  ('node_28_1488772086007', '